# FIFA Scheduling Code

In [ ]:
import numpy as np
from ortools.sat.python import cp_model

In [2]:
teams = [['Argentina', 'Brazil', 'Denmark', 'Peru'],
         ['France', 'Croatia', 'Mexico', 'Poland'],
         ['Egypt', 'Belgium', 'Spain', 'USA'],
         ['Portugal', 'England', 'Morocco', 'Uruguay']]

matchups = []

In [7]:
# Solver code
def schedule_matches(M, no_adjacent_matches, capacities, iteration=0):
    num_matches = len(no_adjacent_matches)
    num_stadiums = len(capacities)
    num_days = len(M[0])
# Create the model
    model = cp_model.CpModel()
# define the variables
    matches = {}
    for m in range(num_matches):
        for d in range(num_days):
            for s in range(num_stadiums):
                matches[(m, d, s)] = model.NewBoolVar(f"match_{m}_day_{d}_stadium_{s}")

    # Constraint: Matches cannot be within 3 days of each other
    for m1 in range(num_matches):
        for m2 in no_adjacent_matches[m1]:
            for d in range(num_days):
                        for prev_d in range(max(0, d - 3), d):
                            if prev_d >= 0:
                                model.Add(matches[(m1, d, s)] + matches[(m2, prev_d, s)] <= 1)
                        for next_d in range(d + 1, min(d + 4, num_days)):
                            model.Add(matches[(m1, d, s)] + matches[(m2, next_d, s)] <= 1)
    
    # Constraint: Each match is scheduled exactly once
    for m in range(num_matches):
        model.Add(sum(matches[(m, d, s)] for d in range(num_days) for s in range(num_stadiums)) == 1)
        
    # Constraint: Each stadium hosts at most one match per day
    for d in range(num_days):
        for s in range(num_stadiums):
            model.Add(sum(matches[(m, d, s)] for m in range(num_matches)) <= 1)
    # Objective function; m+1 is because matches are 0-indexed and we need them to be 1-indexed
    model.Maximize(
        sum(capacities[s] * (m + 1) * M[s][d] * matches[(m, d, s)]
            for m in range(num_matches)
            for d in range(num_days)
            for s in range(num_stadiums))
    )
    # Solve the model and return as a matrix
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        scheduled = [[0 for _ in range(num_days)] for _ in range(num_stadiums)]
        for d in range(num_days):
            for m in range(num_matches):
                for s in range(num_stadiums):
                    if solver.Value(matches[(m, d, s)]) == 1:
                        scheduled[s][d] = m + 1 + iteration #since matches are 1 to 6, 'iteration' adds the required amount to make the matches 7-12, 13-18, 19-14
        return scheduled
    else:
        return None

In [4]:
def update_matrix(M, solution):
    if not solution:
        return None

    for s in range(len(solution)):
        for d in range(len(solution[0])):
            if solution[s][d] != 0:
                M[s][d] = 0
    
    return M

In [19]:
    group_matchups = []
    for i in range(len(group)):
        for j in range(i+1, len(group)):
            group_matchups.append((group[i], group[j]))
    matchups.append(group_matchups)

# Print the matchups for each group
for i, group_matchups in enumerate(matchups):
    print(f"Matchups for Group {i+1}:")
    for matchup in group_matchups:
        print(matchup)


Matchups for Group 1:
('Argentina', 'Brazil')
('Argentina', 'Denmark')
('Argentina', 'Peru')
('Brazil', 'Denmark')
('Brazil', 'Peru')
('Denmark', 'Peru')
Matchups for Group 2:
('France', 'Croatia')
('France', 'Mexico')
('France', 'Poland')
('Croatia', 'Mexico')
('Croatia', 'Poland')
('Mexico', 'Poland')
Matchups for Group 3:
('Egypt', 'Belgium')
('Egypt', 'Spain')
('Egypt', 'USA')
('Belgium', 'Spain')
('Belgium', 'USA')
('Spain', 'USA')
Matchups for Group 4:
('Portugal', 'England')
('Portugal', 'Morocco')
('Portugal', 'Uruguay')
('England', 'Morocco')
('England', 'Uruguay')
('Morocco', 'Uruguay')


As you can see, because of the way the groups were formulated, the `no_adjacent_matches` will be the same for all groups

In [6]:
no_adjacent_matches = [[1,2,3,4],[0,2,3,5],[0,1,4,5],[0,1,4,5],[0,2,3,5],[1,2,3,4]]

In [21]:
M = [
    [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
    [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
    [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
    [0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
    [0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.],
    [0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.]
]
#Stadium capacities for the central region, in the order of the FIFA schedule
capacities = [48000, 83000, 53500, 72000, 94000, 76000]
#Generate a solution, update the matrix M, repeat for the next set of matches until all of them are scheduled.
solution = schedule_matches(M, no_adjacent_matches, capacities, iteration=0)
updated_M = update_matrix(M, solution)
solution2 = schedule_matches(updated_M, no_adjacent_matches, capacities, iteration = 6)
updated_M2 = update_matrix(updated_M, solution2)
solution3 = schedule_matches(updated_M2, no_adjacent_matches, capacities, iteration = 12)
updated_M3 = update_matrix(updated_M2, solution3)
solution4 = schedule_matches(updated_M3, no_adjacent_matches, capacities, iteration = 18)

In [22]:
def combine_matrices(matrix1, matrix2):
    if len(matrix1) != len(matrix2) or len(matrix1[0]) != len(matrix2[0]):
        return None  # Matrices must have the same dimensions
    
    combined_matrix = []
    for i in range(len(matrix1)):
        row = []
        for j in range(len(matrix1[0])):
            row.append(matrix1[i][j] + matrix2[i][j])
        combined_matrix.append(row)
    
    return combined_matrix

part1 = combine_matrices(solution, solution2)
part2 = combine_matrices(solution3, solution4)
total = combine_matrices(part1, part2)

if total:
    print("Combined matrix:")
    for row in total:
        print(row)
else:
    print("Matrices must have the same dimensions!")

Combined matrix:
[20, 0, 0, 0, 0, 0, 0, 21, 0, 0, 0, 0, 22, 0, 0, 19, 0]
[1, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0]
[0, 0, 0, 24, 0, 0, 0, 0, 0, 23, 0, 0, 0, 13, 0, 0, 0]
[0, 0, 0, 17, 0, 0, 15, 0, 0, 16, 0, 0, 18, 0, 0, 14, 0]
[0, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 5, 0, 0, 3, 0, 6]
[0, 0, 0, 0, 0, 7, 0, 0, 0, 8, 0, 0, 0, 0, 10, 0, 9]


In [23]:
matchups_ranked = [('Denmark', 'Peru'),('Brazil', 'Peru'),('Brazil', 'Denmark'),('Argentina', 'Peru'),('Argentina', 'Denmark'),('Argentina', 'Brazil'),
                   ('Mexico', 'Poland'),('Croatia', 'Poland'),('Croatia', 'Mexico'),('France', 'Poland'),('France', 'Mexico'),('France', 'Croatia'),
                   ('Egypt', 'USA'),('Egypt', 'Spain'),('Egypt', 'Belgium'),('Spain', 'USA'),('Belgium', 'USA'),('Belgium', 'Spain'),
                   ('Morocco', 'Uruguay'),('Portugal', 'Uruguay'),('England', 'Uruguay'),('Portugal', 'Morocco'),('England', 'Morocco'),('Portugal', 'England')]

In [24]:
# Fill the matrix with the corresponding matches
for match_num, match in enumerate(matchups_ranked, start=1):
    for i, row in enumerate(total):
        for j, cell in enumerate(row):
            if cell == match_num:
                total[i][j] = f" {match[0]} vs. {match[1]}"

# Print the filled matrix
for row in total:
    print(row)

[' Portugal vs. Uruguay', 0, 0, 0, 0, 0, 0, ' England vs. Uruguay', 0, 0, 0, 0, ' Portugal vs. Morocco', 0, 0, ' Morocco vs. Uruguay', 0]
[' Denmark vs. Peru', 0, 0, 0, 0, 0, ' France vs. Croatia', 0, 0, 0, 0, 0, 0, ' France vs. Mexico', 0, 0, 0]
[0, 0, 0, ' Portugal vs. England', 0, 0, 0, 0, 0, ' England vs. Morocco', 0, 0, 0, ' Egypt vs. USA', 0, 0, 0]
[0, 0, 0, ' Belgium vs. USA', 0, 0, ' Egypt vs. Belgium', 0, 0, ' Spain vs. USA', 0, 0, ' Belgium vs. Spain', 0, 0, ' Egypt vs. Spain', 0]
[0, 0, 0, ' Brazil vs. Peru', 0, 0, ' Argentina vs. Peru', 0, 0, 0, 0, ' Argentina vs. Denmark', 0, 0, ' Brazil vs. Denmark', 0, ' Argentina vs. Brazil']
[0, 0, 0, 0, 0, ' Mexico vs. Poland', 0, 0, 0, ' Croatia vs. Poland', 0, 0, 0, 0, ' France vs. Poland', 0, ' Croatia vs. Mexico']
